In [19]:
import pandas as pd
import numpy as np
import google.auth
from google.auth import compute_engine
from google.cloud import bigquery

# Create the BQ client. This will ask you to log in the first time.

project_id = 'data-engineering-prod'
client = bigquery.Client(project=project_id)

# List all datasets in project
client.list_datasets()

In [22]:
# list datasets and number of views/tables
datasets = []
for dataset in client.list_datasets():
    views = list(client.list_tables(dataset=dataset.reference))
    datasets.append(views)
    print(dataset.dataset_id + " contains {:d} views".format(len(views)))
    
print("\n" + project_id + " contains {:d} datasets".format(len(datasets)))

landing_andromeda contains 107 views
landing_andromeda_secure contains 660 views
landing_identity_secure contains 4 views
landing_orex_secure contains 16 views
landing_orion_migration_secure contains 51 views
landing_orion_secure contains 3 views
landing_pace_secure contains 1 views
landing_payments_secure contains 23 views
product_andromeda_secure contains 195 views
product_identity_secure contains 1 views
product_orion contains 31 views
product_orion_secure contains 99 views
product_payments_secure contains 1 views
raw_andromeda_secure contains 418 views
raw_identity_secure contains 2 views
raw_orex_secure contains 11 views
raw_orion_migration_secure contains 13 views
raw_orion_secure contains 1 views
raw_pace_secure contains 1 views
raw_payments_secure contains 5 views

data-engineering-prod contains 20 datasets


In [ ]:
# Collect data on all views within datasets
print("Collecting data on all views in " + project_id + ". This may take some time...\n")
views = []

for dataset in client.list_datasets():
    print("Collecting views for: " + dataset.dataset_id)
    
    for view in list(client.list_tables(dataset=dataset.reference)):
        try:
            dataset_ref = bigquery.DatasetReference(project_id, dataset.dataset_id)
            view_ref = dataset_ref.table(view.table_id)
            v = client.get_table(view_ref)
            views.append(v)

        except Exception, err:
            print(err)
            pass
        
print("\nDone!")

In [5]:
# Example data from table/view
print("Example table/view and query:")
print("\nView: {}".format(views[0].full_table_id))
print("\nView Query:\n{}".format(views[0].view_query))
print("\nType: {}".format(views[0].table_type))

Example table/view and query:

View: data-engineering-prod:landing_andromeda.energy_contracts_consumption_charge_generated_v2

View Query:
#standardSQL
SELECT
  `kafkaData`,
  `unionRecord`,
  `_PARTITIONTIME` `PARTITIONTIME`
FROM
  `data-engineering-prod.auto_capture_v2.energy_contracts_consumption_charge_generated_v2`

Type: VIEW


In [21]:
data = []
for view in views:
    # if the table is created with a query I am assuming it is a view (https://cloud.google.com/bigquery/docs/view-metadata)
    d = [view.dataset_id, view.table_id, view.table_type, view.view_query, view.location, view.num_bytes, view.modified.strftime('%Y-%m-%d %H:%M:%S'), view.created.strftime('%Y-%m-%d %H:%M:%S'), view.self_link]
    data.append(d)
    
df = pd.DataFrame(data).to_json("views.json")
pd.read_json("views.json")

AttributeError: 'TableListItem' object has no attribute 'view_query'